# Neural Net
// TODO: Write a introduction part here

## Step 1: Data Preprocessing

In this step, we import the project dependencies and data. Typically, we would also perform data cleaning and wrangling here, but since our data is already clean and well-structured, we only ensure that the data types are read correctly.

In [1]:
import csv
import math
import random

# Read CSV file without checking for missing values
data = []
with open("data/full_data.csv", newline="") as csvfile:
    reader = csv.reader(csvfile)
    headers = next(reader)  # Skip the header
    for row in reader:
        # Convert relevant columns to numerical values
        income = float(row[1])
        age = float(row[2])
        loan = float(row[3])
        target = int(row[4])
        # Append the row to the data
        data.append([row[0], income, age, loan, target])

## Step 2: Split Data

In this step, we will split the data into two parts: training data and test data. The training data will be used to teach the model, while the test data will help us check how well the model works on new, unseen data. This way, we can ensure that the model is not just memorizing the training data but is actually learning patterns that can be applied in real situations.

A model can "memorize" the data if it’s too complex or if it’s trained for too long on a small dataset. This is called overfitting. When overfitting happens, the model performs very well on the training data but struggles with new, unseen data.

In [4]:
# Shuffle the dataset
random.seed(42)  # For reproducibility
random.shuffle(data)

# Define the split ratio (80% training, 20% testing)
split_ratio = 0.8
split_index = int(len(data) * split_ratio)


# Split the data into training and testing sets
train_data = data[:split_index]
test_data = data[split_index:]


# Save the training data to a CSV file
with open("data/train_data.csv", "w", newline="") as trainfile:
    writer = csv.writer(trainfile)
    writer.writerow(headers)  # Write the header
    writer.writerows(train_data)  # Write the training data


# Save the testing data to a CSV file
with open("data/test_data.csv", "w", newline="") as testfile:
    writer = csv.writer(testfile)
    writer.writerow(headers)  # Write the header
    writer.writerows(test_data)  # Write the testing data


## 3. Feature Extraction and Target Variable Selection

In this step, we extract the relevant columns from the dataset. The columns `income`, `age`, and `loan` are selected as features, which are stored in `X_train` and `X_test`. These features will be used as input to train and evaluate the model.

Additionally, we select the `class` column as the target variable, which is stored in `y_train` and `y_test`. The target variable represents the outcome or category we want the model to predict.

In [3]:
# Extract features and target from training data
X_train = [[row[1], row[2], row[3]] for row in train_data]  # income, age, loan
y_train = [row[4] for row in train_data]  # class (target)

# Extract features and target from testing data
X_test = [[row[1], row[2], row[3]] for row in test_data]  # income, age, loan
y_test = [row[4] for row in test_data]  # class (target)


# 4. Min-Max Scaling

In this step, we will normalize the features of our dataset using Min-Max scaling. This technique adjusts each feature so that its values range between 0 and 1. By doing this, we ensure that all features contribute equally to the model's learning process, preventing features with larger value ranges from overshadowing those with smaller ranges.


The formula for the min-max function is:
### $x_{scaled} = \frac{x - x_{min}}{x_{max} - x_{min}}$


In [6]:
# Manually normalize the data using Min-Max scaling
def min_max_scaling(X):
    min_vals = [min(col) for col in zip(*X)]
    max_vals = [max(col) for col in zip(*X)]

    X_scaled = []
    for row in X:
        scaled_row = [
            (row[i] - min_vals[i]) / (max_vals[i] - min_vals[i])
            for i in range(len(row))
        ]
        X_scaled.append(scaled_row)

    return X_scaled


X_train_scaled = min_max_scaling(X_train)
X_test_scaled = min_max_scaling(X_test)

# 5. Sigmoid & Sigmoid Derivative
The sigmoid function is an activation function used in neural networks to introduce non-linearity. It maps any input Z (the weighted sum of inputs to a neuron) to a value between 0 and 1. The function's output can be interpreted as the probability of belonging to class 1. Values close to 0 represent class 0, while values close to 1 represent class 1, creating a clear decision boundary.

The formula for the sigmoid function is:
### $\sigma(z) = \frac{1}{1 + e^{-z}}$

The derivative of the sigmoid function is important for backpropagation in neural networks, as it helps calculate the gradient for adjusting the weights during training:
### $\sigma'(z) = \sigma(z) \cdot (1 - \sigma(z))$
This derivative shows how the output of the sigmoid changes with respect to the input, which is essential for optimizing the model.

In [7]:

# Sigmoid function
def sigmoid(z):
    return 1 / (1 + math.exp(-z))


# Derivative of the sigmoid function
def sigmoid_derivative(a):
    return a * (1 - a)

# 5. Initialize Weights and Biases
In this step, we initialize the weights and biases for our neural network. This is a crucial step as it sets the starting point for the training process.

1. **Function to Initialize Random Weights:**
   - We define a function `random_matrix(rows, cols)` that generates a matrix of random values with the specified number of rows and columns. This function does not use NumPy and relies on Python's built-in `random` module.

2. **Define Network Architecture:**
   - `input_size`: The number of input features, which is determined by the length of the first row in `X_train_scaled`. In this case, we have 3 input features: income, age, and loan.
   - `hidden_size`: The number of neurons in the hidden layer. We set this to 4.
   - `output_size`: The number of output neurons. We set this to 1, as we are predicting a single value.

3. **Initialize Weights and Biases:**
   - `W1`: A matrix of random weights connecting the input layer to the hidden layer. It has dimensions `input_size x hidden_size`.
   - `b1`: A list of random biases for the hidden layer. It has a length of `hidden_size`.
   - `W2`: A matrix of random weights connecting the hidden layer to the output layer. It has dimensions `hidden_size x output_size`.
   - `b2`: A list of random biases for the output layer. It has a length of `output_size`.

By initializing the weights and biases randomly, we ensure that the neural network starts with a diverse set of parameters, which helps in breaking symmetry and allows the network to learn effectively during training.

In [8]:
# Function to initialize random weights without using numpy
def random_matrix(rows, cols):
    return [[random.random() for _ in range(cols)] for _ in range(rows)]


# Initialize weights and biases
input_size = len(X_train_scaled[0])  # 3 input features: income, age, loan
hidden_size = 4
output_size = 1


# Randomly initialize weights and biases
W1 = random_matrix(input_size, hidden_size)
b1 = [random.random() for _ in range(hidden_size)]
W2 = random_matrix(hidden_size, output_size)
b2 = [random.random() for _ in range(output_size)]

In [9]:
# Matrix multiplication function (for dot product)
def matrix_multiply(A, B):
    return [
        [sum(a * b for a, b in zip(A_row, B_col)) for B_col in zip(*B)] for A_row in A
    ]


In [10]:
# Adding bias to a matrix
def add_bias(matrix, bias):
    return [
        [matrix[row][col] + bias[col] for col in range(len(bias))]
        for row in range(len(matrix))
    ]

In [11]:
# Element-wise application of the sigmoid function
def apply_sigmoid(matrix):
    return [[sigmoid(x) for x in row] for row in matrix]


In [12]:
# Forward propagation
def forward(X, W1, b1, W2, b2):
    Z1 = add_bias(matrix_multiply(X, W1), b1)  # Input to hidden layer
    A1 = apply_sigmoid(Z1)  # Activation in hidden layer
    Z2 = add_bias(matrix_multiply(A1, W2), b2)  # Input to output layer
    A2 = apply_sigmoid(Z2)  # Final output (prediction)
    return A1, A2

In [13]:
# Backpropagation (for one epoch)
def backprop(X, y, W1, b1, W2, b2, A1, A2, learning_rate=0.1):
    m = len(y)

    dZ2 = [[A2[i][0] - y[i]] for i in range(m)]  # Derivative of loss w.r.t output
    dW2 = [
        [sum(A1[i][h] * dZ2[i][0] for i in range(m)) / m for _ in range(output_size)]
        for h in range(hidden_size)
    ]
    db2 = [sum(dZ2[i][0] for i in range(m)) / m]

    dA1 = [
        [
            sum(W2[h][o] * dZ2[i][0] for o in range(output_size))
            for h in range(hidden_size)
        ]
        for i in range(m)
    ]
    dZ1 = [
        [dA1[i][h] * sigmoid_derivative(A1[i][h]) for h in range(hidden_size)]
        for i in range(m)
    ]
    dW1 = [
        [sum(X[i][f] * dZ1[i][h] for i in range(m)) / m for h in range(hidden_size)]
        for f in range(input_size)
    ]
    db1 = [sum(dZ1[i][h] for i in range(m)) / m for h in range(hidden_size)]

    # Update weights and biases using gradient descent
    W1 = [
        [W1[f][h] - learning_rate * dW1[f][h] for h in range(hidden_size)]
        for f in range(input_size)
    ]
    b1 = [b1[h] - learning_rate * db1[h] for h in range(hidden_size)]
    W2 = [
        [W2[h][o] - learning_rate * dW2[h][o] for o in range(output_size)]
        for h in range(hidden_size)
    ]
    b2 = [b2[o] - learning_rate * db2[o] for o in range(output_size)]

    return W1, b1, W2, b2


In [14]:
# Training loop
for epoch in range(10000):  # Number of epochs
    A1, A2 = forward(X_train_scaled, W1, b1, W2, b2)
    W1, b1, W2, b2 = backprop(X_train_scaled, y_train, W1, b1, W2, b2, A1, A2)

    # Optional: Calculate loss for monitoring
    if epoch % 1000 == 0:
        loss = sum(
            -y_train[i] * math.log(A2[i][0]) - (1 - y_train[i]) * math.log(1 - A2[i][0])
            for i in range(len(y_train))
        ) / len(y_train)
        print(f"Epoch {epoch}, Loss: {loss}")


Epoch 0, Loss: 2.3317760965068706
Epoch 1000, Loss: 0.39258092513604625
Epoch 2000, Loss: 0.3791446938451353
Epoch 3000, Loss: 0.34456667813829983
Epoch 4000, Loss: 0.29808416957613454
Epoch 5000, Loss: 0.25433260579048994
Epoch 6000, Loss: 0.21775096391199814
Epoch 7000, Loss: 0.19001783867416813
Epoch 8000, Loss: 0.1700869721003493
Epoch 9000, Loss: 0.15602323353907965


In [15]:
# Save the weights and biases after training
def save_weights_biases(W, b, W_file, b_file):
    with open(W_file, "w") as f_w, open(b_file, "w") as f_b:
        for row in W:
            f_w.write(",".join(map(str, row)) + "\n")
        f_b.write(",".join(map(str, b)) + "\n")


save_weights_biases(W1, b1, "model_weights/W1.csv", "model_weights/b1.csv")
save_weights_biases(W2, b2, "model_weights/W2.csv", "model_weights/b2.csv")

In [16]:
# Testing the model
A1_test, A2_test = forward(X_test_scaled, W1, b1, W2, b2)
predictions = [1 if a > 0.5 else 0 for a in [row[0] for row in A2_test]]
accuracy = sum([1 for i in range(len(y_test)) if predictions[i] == y_test[i]]) / len(
    y_test
)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.5625
